In [1]:
import numpy as np

In [8]:
raw_text = open('cien sonetos de amor.txt').readlines()

In [22]:
raw_text[:20]

['Soneto I \n',
 '\n',
 'Matilde, nombre de planta o piedra o vino, \n',
 'de lo que nace de la tierra y dura, \n',
 'palabra en cuyo crecimiento amanece, \n',
 'en cuyo estío estalla la luz de los limones. \n',
 'En ese nombre corren navíos de madera \n',
 'rodeados por enjambres de fuego azul marino, \n',
 'y esas letras son el agua de un río \n',
 'que desemboca en mi corazón calcinado. \n',
 'Oh nombre descubierto bajo una enredadera \n',
 'como la puerta de un túnel desconocido \n',
 'que comunica con la fragancia del mundo! \n',
 'Oh invádeme con tu boca abrasadora, \n',
 'indágame, si quieres, con tus ojos nocturnos, \n',
 'pero en tu nombre déjame navegar y dormir.\n',
 '\n',
 ' \n',
 '\n',
 'Soneto II \n']

In [155]:
raw_text_no_cr = []
for i, line in enumerate(raw_text):
    if line != '\n' and line!=' \n':
        if line[:7] == 'Soneto ':
            if i>0: raw_text_no_cr.append(' END_SONETO ')
            raw_text_no_cr.append(' START_SONETO ')
        else:
            raw_text_no_cr.append(line.replace(' \n', '\n')
                                  .replace('«', '"')
                                  .replace('»', '"')
                                  .replace('-', 'char_guion_medio')
                                  .replace('(', 'char_open_parent')
                                  .replace(')', 'char_close_parent')
                                  .replace('\n', ' char_new_line ')
                                  .replace('.', ' char_dot ')
                                  .replace(',', ' char_comma ')
                                  .replace('¡', ' char_exclamation_open ')
                                  .replace('!', ' char_exclamation_close ')
                                  .replace('¿', ' char_question_open ')
                                  .replace('?', ' char_question_close ')
                                  .replace(':', ' char_dos_puntos ')
                                  .replace('"', ' char_comillas ')
                                 )

In [156]:
raw_text_no_cr[:50]

[' START_SONETO ',
 'Matilde char_comma  nombre de planta o piedra o vino char_comma  char_new_line ',
 'de lo que nace de la tierra y dura char_comma  char_new_line ',
 'palabra en cuyo crecimiento amanece char_comma  char_new_line ',
 'en cuyo estío estalla la luz de los limones char_dot  char_new_line ',
 'En ese nombre corren navíos de madera char_new_line ',
 'rodeados por enjambres de fuego azul marino char_comma  char_new_line ',
 'y esas letras son el agua de un río char_new_line ',
 'que desemboca en mi corazón calcinado char_dot  char_new_line ',
 'Oh nombre descubierto bajo una enredadera char_new_line ',
 'como la puerta de un túnel desconocido char_new_line ',
 'que comunica con la fragancia del mundo char_exclamation_close  char_new_line ',
 'Oh invádeme con tu boca abrasadora char_comma  char_new_line ',
 'indágame char_comma  si quieres char_comma  con tus ojos nocturnos char_comma  char_new_line ',
 'pero en tu nombre déjame navegar y dormir char_dot  char_new_line ',


In [157]:
all_text = ''.join(raw_text_no_cr)
print(all_text[:200])

 START_SONETO Matilde char_comma  nombre de planta o piedra o vino char_comma  char_new_line de lo que nace de la tierra y dura char_comma  char_new_line palabra en cuyo crecimiento amanece char_comma


In [158]:
from sklearn.feature_extraction.text import CountVectorizer

In [159]:
vectorizer = CountVectorizer(stop_words=None, token_pattern='(?u)\\b\\w+\\b')

In [160]:
matrix = vectorizer.fit_transform([all_text])

In [161]:
len(vectorizer.vocabulary_)

2568

In [162]:
import re

In [163]:
separators = re.compile('(?u)\\b\\w+\\b').split(all_text)

In [164]:
for s in separators:
    if s!=' ' and s!='  ':
        print(s)

In [190]:
words = re.findall('(?u)\\b\\w+\\b', all_text)

In [195]:
words[:5]

['START_SONETO', 'Matilde', 'char_comma', 'nombre', 'de']

In [200]:
coded_words = [vectorizer.vocabulary_[word.lower()] for word in words]

In [202]:
coded_words[:10]

[2201, 1458, 425, 1608, 660, 1789, 1627, 1769, 1627, 2502]

In [212]:
len(set(coded_words))

2568

In [208]:
from keras.utils import to_categorical

In [210]:
to_categorical(coded_words).shape

(13990, 2568)

In [219]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [222]:
model = Sequential()
model.add(Embedding(len(set(coded_words)), 64))
model.add(LSTM(100))
model.add(Dense(len(set(coded_words)), activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 64)          164352    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_3 (Dense)              (None, 2568)              259368    
Total params: 489,720
Trainable params: 489,720
Non-trainable params: 0
_________________________________________________________________
